In [1]:
import os
from rdkit.Chem import AllChem
from rdkit import Chem
import torch
import numpy as np
from torch_geometric.data import Data

# from torch.utils.data import DataLoader

# from torch_geometric.loader.dataloader import DataLoader
from torch_geometric.data import DataLoader
# from dataloader import DataLoaderSubstructContext

from model import GNN, GNN_graphpred
from loader import CustomDataset

from tqdm import tqdm

# from torchsummary import summary

/raid/cs21mtech11009/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# torch.autograd.set_detect_anomaly(True)

In [4]:
device = torch.device("cuda:3")
print(device)

cuda:3


In [5]:
dataset = CustomDataset(root="data")
loader = DataLoader(dataset, batch_size=1, shuffle=False)

/raid/cs21mtech11009/.local/lib/python3.6/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
# model = GNN(num_layer=5, emb_dim=300).to(device)
# model.load_state_dict(torch.load("../chem/model_gin/contextpred.pth"))

In [7]:
model = GNN_graphpred(num_layer=5, emb_dim=300, num_tasks=1, JK="last", drop_ratio=0.5, graph_pooling="mean")
model.from_pretrained("../chem/model_gin/supervised.pth")
model.to(device)

GNN_graphpred(
  (gnn): GNN(
    (x_embedding1): Embedding(120, 300)
    (x_embedding2): Embedding(3, 300)
    (gnns): ModuleList(
      (0): GINConv()
      (1): GINConv()
      (2): GINConv()
      (3): GINConv()
      (4): GINConv()
    )
    (batch_norms): ModuleList(
      (0): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (graph_pred_linear): Linear(in_features=300, out_features=1, bias=True)
)

In [8]:
# loader = DataLoaderSubstructContext(dataset, batch_size=1, shuffle=True)
model.eval()
for step, batch in enumerate(tqdm(loader, desc="Iteration")):
#     print(1, batch.x, batch.y)
    batch = batch.to(device)
    print(batch.x.shape)
    with torch.no_grad():
        y = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
# print(dir(model))
# summary(model)

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([6, 2])


Iteration:   0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered